In [ ]:
import requests,time,datetime
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [ ]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [ ]:
df = df_hsA

In [ ]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2019')

ss.getStockDataByType('E:/wangtao/PythonWorkSpace/DataAnalysis/StockAnalyze/data/')
# ss.getStockDataByType('E:/PythonWorkSpace/DataAnalysis/Stock_Companies/data/')


In [ ]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [ ]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})
df_2019 = pd.read_excel("data/2019年报.xls",converters= {u'stock_code':str})

In [ ]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于15%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [ ]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [ ]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [ ]:
df_target.shape

In [ ]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') &(x != '亏损') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_S']) / x['PE_M']),axis=1)
df_target = df_target[df_target['pe_rate'] < 0.6]

In [ ]:
df_target.shape

In [ ]:
df_target.sort_values(by=['pe_rate'],ascending=True)